Fau Topics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Thesis/

/content/drive/MyDrive/Thesis


In [ ]:
!pip install bertopic

In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [ ]:
! python -m spacy download de_dep_news_trf

2023-11-27 17:21:29.645193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 17:21:29.645282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 17:21:29.645335: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 17:21:31.146519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.8/412.8 MB 942.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 17.7 MB/s eta 0

In [ ]:
import spacy
import de_dep_news_trf
import pandas as pd
from bertopic import BERTopic
from pandas import read_excel

In [ ]:
nlp = spacy.load('de_dep_news_trf')

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('german'))
print(stops)

{'sind', 'dort', 'können', 'welche', 'würde', 'von', 'werde', 'ins', 'und', 'eures', 'habe', 'kann', 'nichts', 'werden', 'sehr', 'jetzt', 'ich', 'oder', 'um', 'daß', 'demselben', 'eines', 'ihm', 'ihnen', 'wo', 'dann', 'selbst', 'muss', 'bist', 'diesen', 'anderem', 'ohne', 'keine', 'seinem', 'ander', 'einig', 'für', 'euren', 'haben', 'hatte', 'diese', 'keinem', 'sein', 'jedem', 'wenn', 'unsere', 'jedes', 'jenen', 'meine', 'zwar', 'meiner', 'ihrem', 'er', 'hat', 'unserem', 'der', 'anderr', 'hinter', 'denselben', 'jenes', 'seines', 'zum', 'soll', 'aber', 'was', 'allen', 'einem', 'einen', 'anderes', 'anderm', 'mit', 'machen', 'waren', 'eurer', 'kein', 'gewesen', 'das', 'keinen', 'meinen', 'dass', 'deinem', 'nun', 'musste', 'anderen', 'weil', 'wie', 'euch', 'dieselben', 'einiger', 'deine', 'mein', 'ist', 'doch', 'ihren', 'aller', 'wieder', 'wollte', 'deiner', 'manches', 'aus', 'sich', 'will', 'da', 'ob', 'etwas', 'euer', 'meinem', 'dir', 'hier', 'hin', 'hatten', 'derselben', 'ihr', 'derselb

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#prepare data
df = pd.read_excel("Reflection_Data.xlsx")
fau=df['Text'].values.tolist()
normalized=[]
for text in fau:
    doc = nlp(text)
    # sentence segmentation
    for sent in doc.sents:
      segmented=' '.join([token.lemma_ for token in sent if token.is_punct==False and str(token).lower() not in stops and token.is_alpha==True])
      normalized.append(segmented)
with open('sentences_fau2.txt','w', encoding='utf-8') as outpy:
    for e in normalized:
        outpy.write(e+'\n')

In [ ]:
print(len(normalized))

14051


In [ ]:
with open('sentences_fau2.txt','r',encoding='utf-8') as fau:
    fau=list(fau)
    print(len(fau))
    fau=[e.replace('\n','') for e in fau]

14051


In [ ]:
print(len(fau))
fau=[e.lower() for e in fau if e!='']
print(len(fau))

14051
13491


In [ ]:
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import MaximalMarginalRelevance

representation_model_1 = MaximalMarginalRelevance(diversity=0.2)
representation_model_2 = KeyBERTInspired()
representation_models = [representation_model_1, representation_model_2]

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model = BERTopic(embedding_model="paraphrase-multilingual-mpnet-base-v2",
                       nr_topics="auto",ctfidf_model=ctfidf_model,representation_model=representation_models,
                       language="German",min_topic_size=60)


topics, probabilities = topic_model.fit_transform(fau)

In [ ]:
count_df = topic_model.get_topic_info()
count_df['Perc'] = (count_df['Count'] / count_df['Count'].sum()).round(2)

df = count_df[['Topic','Name','Count','Perc']]
df


,Topic,Name,Count,Perc
0,-1,-1_verständlich_reflexion_verstehen_anwenden,5264,0.39
1,0,0_auseinandersetzen_ausführlich_verständlich_v...,2873,0.21
2,1,1_unterrichtsgestaltung_unterrichten_unterrich...,1010,0.07
3,2,2_diagnostik_diagnostisch_diagnostizieren_päda...,756,0.06
4,3,3_zeitaufwand_zeitdruck_konzentrieren_konzentr...,392,0.03
5,4,4_klassenmanagement_klassenmanagment_klassenfü...,353,0.03
6,5,5_zufriedenstellend_zufrieden_lernergebnis_end...,233,0.02
7,6,6_feedbackdatei_feedback_feedbackdokument_feed...,214,0.02
8,7,7_deutschklasse_deutsch_themenbereich_brennpun...,194,0.01
9,8,8_diagnoseverfahren_diagnostisch_diagnostik_di...,185,0.01


In [ ]:
topic_model.save("/content/drive/MyDrive/Thesis/fau_topics_best")

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
topic_model_fau = BERTopic.load("/content/drive/MyDrive/Thesis/fau_topics_best")

In [ ]:
topic_model_fau.visualize_hierarchy()

In [ ]:
topic_model_fau.visualize_heatmap()

In [ ]:
#topics to merge 0 and 19,23 - similarilty 86%
#topics to merge 1 and 4 - similarilty 85%
#topics to merge 2 and 8 - similarilty 83%
#topics to merge 3 and 22 - similarilty 83%
#topics to merge 13 and 24 - similarilty 81%
#topics to merge 15 and 20 - similarilty 85%

topics_to_merge = [[0,19,23],[1, 4], [2, 8], [3,22],[13,24],[15,20]]
topic_model_fau.merge_topics(fau, topics_to_merge)

In [ ]:
count_df = topic_model_fau.get_topic_info()
count_df['Perc'] = (count_df['Count'] / count_df['Count'].sum()).round(2)
df = count_df[['Topic','Name','Count','Perc']]
df

,Topic,Name,Count,Perc
0,-1,-1_verständlich_reflexion_verstehen_anwenden,5264,0.39
1,0,0_auseinandersetzen_beispiel_selbststudium_aus...,3054,0.23
2,1,1_klassenmanagment_klassenmanagement_unterrich...,1363,0.10
3,2,2_pädagogischen_pädagogisch_pädagogische_diagn...,941,0.07
4,3,3_zeitaufwand_bearbeitungszeit_zeitdruck_aufga...,475,0.04
5,4,4_teilfrage_zweiter_fragebeantwortung_vorlegen,234,0.02
6,5,5_zufriedenstellend_zufrieden_lernergebnis_erg...,233,0.02
7,6,6_feedbackdatei_feedbackvorschlag_feedback_fee...,214,0.02
8,7,7_strukturieren_zusammenfassungung_strukturier...,208,0.02
9,8,8_deutschklasse_brennpunktschule_deutsch_theme...,194,0.01


In [ ]:
topic_model_fau.visualize_heatmap()

In [ ]:
#topic to merge 2 and 11 - similarilty 80%
#topic to merge 3 and 13 - similarilty 78%
#topic to merge 4 and 7 - similarilty 80%
#topic to merge 7 and 18 - similarilty 77%
#topic to merge 4 and 18 - similarilty 76%
#topic to merge 4 and 14 - similarilty 75% but with others it is less

topics_to_merge = [[2,11],[3,13], [4,7,18]]
topic_model_fau.merge_topics(fau, topics_to_merge)

In [ ]:
count_df = topic_model_fau.get_topic_info()
count_df['Perc'] = (count_df['Count'] / count_df['Count'].sum()).round(2)
df = count_df[['Topic','Name','Count','Perc']]
df

,Topic,Name,Count,Perc
0,-1,-1_verständlich_reflexion_beispiel_anwenden,5264,0.39
1,0,0_auseinandersetzen_beispiel_selbststudium_aus...,3054,0.23
2,1,1_klassenmanagment_klassenmanagement_unterrich...,1363,0.10
3,2,2_pädagogischen_pädagogisch_pädagogische_diagn...,1102,0.08
4,3,3_konzentrieren_konzentriert_bearbeitungszeit_...,614,0.05
5,4,4_verständlich_strukturieren_stichpunkt_teilau...,505,0.04
6,5,5_zufriedenstellend_zufrieden_lernergebnis_erg...,233,0.02
7,6,6_feedbackdatei_feedback_feedbackpapier_feedba...,214,0.02
8,7,7_deutschklasse_brennpunktschule_deutsch_theme...,194,0.01
9,8,8_literaturbezug_literaturübersicht_literaturv...,183,0.01


In [ ]:
df = topic_model_fau.get_document_info(fau)
new_df = df[['Document','Name']]
new_df

,Document,Name
0,zuerst aufgabe fallbeispiel durchlesen überleg...,-1_verständlich_reflexion_beispiel_anwenden
1,anschließend gehen vorlesungsfolie verschaffen...,0_auseinandersetzen_beispiel_selbststudium_aus...
2,darauf schreiben wichtig inhalt bearbeitung au...,-1_verständlich_reflexion_beispiel_anwenden
3,analysieren fallbeispiel notiz prüfen vorlesun...,-1_verständlich_reflexion_beispiel_anwenden
4,danach legen notiz neben,0_auseinandersetzen_beispiel_selbststudium_aus...
...,...,...
13486,inhalt verstehen,-1_verständlich_reflexion_beispiel_anwenden
13487,kurz punkt bringen,-1_verständlich_reflexion_beispiel_anwenden
13488,anhand lerninhalt entwickeln,-1_verständlich_reflexion_beispiel_anwenden
13489,unsicher nutzen reflexion,-1_verständlich_reflexion_beispiel_anwenden


In [ ]:
data = new_df.rename(columns={"Document": "Sentences", "Name": "Topic Labels"})

In [ ]:
data.to_excel("fau.xlsx", index=False)